In [28]:
# import lib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.datasets import make_moons,make_classification,make_regression,make_circles
import sys
from mpl_toolkits.mplot3d import Axes3D 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA,KernelPCA
np.random.seed(23)
np.set_printoptions(precision=3,suppress=True)
plt.style.use('seaborn')

import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.logging.set_verbosity(tf.logging.ERROR)
from tensorflow.examples.tutorials.mnist import input_data

def _sym_decorrelation(W):
    """ Symmetric decorrelation
    i.e. W <- (W * W.T) ^{-1/2} * W
    """
    s, u = np.linalg.eigh(np.dot(W, W.T))
    # u (resp. s) contains the eigenvectors (resp. square roots of
    # the eigenvalues) of W * W.T
    return np.dot(np.dot(u * (1./(np.sqrt(s+1e-8))+1e-8), u.T), W)

In [29]:
# import data
mnist = input_data.read_data_sets('../../Dataset/MNIST/', one_hot=True)
train_data,train_label,val_data,val_label,test_data,test_label = mnist.train.images,mnist.train.labels,mnist.validation.images,mnist.validation.labels,mnist.test.images, mnist.test.labels
print(train_data.shape,train_label.shape,train_data.max(),train_data.min())
print(val_data.shape,val_label.shape,val_data.max(),val_data.min())
print(test_data.shape,test_label.shape,test_data.max(),test_data.min())

Extracting ../../Dataset/MNIST/train-images-idx3-ubyte.gz
Extracting ../../Dataset/MNIST/train-labels-idx1-ubyte.gz
Extracting ../../Dataset/MNIST/t10k-images-idx3-ubyte.gz
Extracting ../../Dataset/MNIST/t10k-labels-idx1-ubyte.gz
(55000, 784) (55000, 10) 1.0 0.0
(5000, 784) (5000, 10) 1.0 0.0
(10000, 784) (10000, 10) 1.0 0.0


In [38]:
# start session 
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

C:\ProgramData\Miniconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning:

An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).



In [31]:
#  create class
def arctan(x):  return tf.arctan(x)
def tan(x)   :  return tf.tan(x)
def d_arctan(x):return 1/(1+x**2)
def d_tanh(x):  return 1/tf.cos(x)

class FNN():
    
    def __init__(self,input,output,act,d_act):
        self.w = tf.Variable(tf.random.normal())
        self.mw,self.mv = np.zeros_like()
        
        self.a = np.ones(output)
        self.b = np.ones(output)
        self.c = np.zeros(output)
        
        self.act  = act
        self.d_act= d_act

In [139]:
def tf_arctan(x): return tf.atan(x)
def tf_tan(x)   : return tf.tan(x)

def d_tf_arctan(x): return 1/(1+x**2)
def d_tf_tan(x):    return 1/(tf.cos(x)**2)

class PCA_Layer():

    def __init__(self,inc,outc,act=tf_arctan,d_act=d_tf_arctan):
        
        if outc == 1:
            self.w = self.norm(tf.Variable(tf.random_normal([inc,outc],stddev=1,seed=2)))
        else:
            self.w = tf.Variable(tf.random_normal([inc,outc],stddev=0.05,seed=2))
        self.a = tf.Variable(tf.ones([outc]))
        self.b = tf.Variable(tf.ones([outc]))
        self.c = tf.Variable(tf.zeros([outc]))
        
        self.mw,self.vw = tf.Variable(tf.zeros_like(self.w)),tf.Variable(tf.zeros_like(self.w))
        self.ma,self.va = tf.Variable(tf.zeros_like(self.a)),tf.Variable(tf.zeros_like(self.a))
        self.mb,self.vb = tf.Variable(tf.zeros_like(self.b)),tf.Variable(tf.zeros_like(self.b))
        self.mc,self.vc = tf.Variable(tf.zeros_like(self.c)),tf.Variable(tf.zeros_like(self.c))
        
        self.act,self.d_act = act,d_act

    def feedforward(self,input=None):
        self.input  = input
        self.layer  = tf.matmul(self.input,self.w) 
        self.layerA = self.a*self.act(self.b*self.layer) + self.c
        return self.layerA
    
    def backprop_first(self):
        pass 

    def backprop(self,gradient=None,which_reg=0):
        grad_part_1 = gradient
        grad_part_2 = self.d_act(self.layer)
        grad_part_3 = self.input

        grad_middle = grad_part_1 * grad_part_2
        grad  = tf.matmul(tf.transpose(grad_part_3),grad_middle)/batch_size
        grad_b= tf.reduce_mean(grad_middle,axis=0)
        grad_pass = tf.matmul(grad_middle,tf.transpose(self.w))

        update_w = []

        # Update the Weight First
        update_w.append(tf.assign( self.m,self.m*beta1 + (1-beta1) * (grad)   ))
        update_w.append(tf.assign( self.v,self.v*beta2 + (1-beta2) * (grad ** 2)   ))
        m_hat = self.m / (1-beta1)
        v_hat = self.v / (1-beta2)
        adam_middle = m_hat *  learning_rate/(tf.sqrt(v_hat) + adam_e)
        update_w.append(tf.assign(self.w,tf.subtract(self.w,adam_middle )))

        return grad_pass,update_w
    
    def norm(self,W):
        W = W/(tf.sqrt(tf.reduce_sum(W**2)+1e-8))
        return W
    
    def sym_decorrelation(self,W):
        s, u = tf.linalg.eigh(W@tf.transpose(W))
        result = (u * (1./(tf.sqrt(s+1e-8)+1e-8)) @ tf.transpose(u) @ W
        return result


In [140]:
temp = np.ones((10,784)).astype(np.float32)
               
layer1 = PCA_Layer(784,256)
sess.run(tf.global_variables_initializer())
te = layer1.w.eval()


In [145]:
te.shape

s, u = tf.linalg.eigh(te@te.T)
print(s.eval())
print(u.eval())

[-0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.
 -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.
 -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.
 -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.
 -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.
 -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.
 -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.
 -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.
 -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.
 -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.
 -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.
 -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.
 -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.
 -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.
 -0.    -0.    -0.    -0.    -0.    -0.    -0.  

In [156]:
ss  =np.random.randn(784,256)
def _sym_decorrelation(W):
    """ Symmetric decorrelation
    i.e. W <- (W * W.T) ^{-1/2} * W
    """
    s, u = np.linalg.eigh(np.dot(W, W.T))
    # u (resp. s) contains the eigenvectors (resp. square roots of
    # the eigenvalues) of W * W.T
    return (u * (1./(np.sqrt(s+1e-8)+1e-8))) @ u.T @ W

sss = _sym_decorrelation(ss)
print(sss.T@sss)

[[ 1. -0. -0. ... -0. -0.  0.]
 [-0.  1. -0. ...  0. -0.  0.]
 [-0. -0.  1. ... -0.  0.  0.]
 ...
 [-0.  0. -0. ...  1.  0. -0.]
 [-0. -0.  0. ...  0.  1.  0.]
 [ 0.  0.  0. ... -0.  0.  1.]]
